# Laboratory 1: Property Graphs
### Luis Alfredo Leon Villapún
### Liliia Aliakberova

# Part D Graph Algorithms
* * *
In this section we will explode the power of graphs by using some of the suggested algorithms in Neo4j.

## Creating the connector
As in previous parts, let's first create the connector to handle the messages with Neo4j.

In [1]:
from connector import Neo4jConnector
from getpass import getpass

uri = "neo4j://localhost:7687"
user = "neo4j"
password = getpass("Input your password to connect")
conn = Neo4jConnector(uri, user, password)

Input your password to connect········


## Triggering algorithms from the graph-data-science library

The graph data science library helps us execute common graph algorithms with Neo4j. This can be useful to analyze our graphs from this point of view, more like a graph rather than a database, providing useful information.

### Betweeness algorithm

This is a very interesting algorithm that provides information about the "bridges" on a particular graph. This means, it will return nodes that have key connections in the graph, connection different regions accross it.

In [6]:
# From documentation site: https://neo4j.com/docs/python-manual/current/

def betweeness_def(tx):
    result = tx.run(
        """
        CALL gds.graph.project('betweeness', 'Paper', {CITED_BY: {properties: 'year'}})
        """
    )
    records = list(result)
    summary = result.consume()
    return records, summary

try:
    with conn.driver.session(database="neo4j") as session:
        records, summary = session.execute_read(betweeness_def)
        print(records, summary)
except Exception as e:
    print("Seems like the graph was already saved on the catalogue.")


def betweeness(tx):
    result = tx.run(
        """
            CALL gds.betweenness.stream('betweeness')
            YIELD nodeId, score
            RETURN gds.util.asNode(nodeId).Title AS name, score
            ORDER BY name ASC
        """
    )
    records = list(result)
    summary = result.consume()
    return records, summary

with conn.driver.session(database="neo4j") as session:
    records, summary = session.execute_read(betweeness)
    print(records, summary)

Seems like the graph was already saved on the catalogue.
[<Record name='3D hydrodynamic focusing for optofluidics using a stacked channel design' score=5705.54744154493>, <Record name='3D printed microfluidic selectable ratio mixer pump in 2 mm3' score=1173.1705085360677>, <Record name='3D printing for lab-on-a-chip devices with 20 _m channels' score=306.9292217592872>, <Record name='A 0.2-V 10-bit 5-kHz SAR ADC with Dynamic Bulk Biasing and Ultra-Low-Supply-Voltage Comparator' score=1697.8802685859719>, <Record name='A Class of Logistic Functions for Approximating State-Inclusive Koopman Operators' score=427.73332657409645>, <Record name='A Geometrically Exact Isogeometric Blended Shell: Formulation, Benchmarking, and Automotive Application' score=283.0864283674282>, <Record name='A Method for Creating Product Social Impact Models of Engineered Products' score=745.5891353242514>, <Record name="A Multimodal Exploration of Engineering Students' Emotions and Electrodermal Activity in Des

### Node similarity algorithm
In this case, we are analysing the similarity between two "Paper" nodes over the edge "RELATED_TO".

In [7]:
# From documentation site: https://neo4j.com/docs/python-manual/current/

def node_simil_def(tx):
    result = tx.run(
        """
        CALL gds.graph.project(
            'NSPaperTopic',
            ['Paper', 'Topic'],
            {
              RELATED_TO: {}
            }
        );
        """
    )
    records = list(result)
    summary = result.consume()
    return records, summary

try:
    with conn.driver.session(database="neo4j") as session:
        records, summary = session.execute_read(node_simil_def)
        print(records, summary)
except Exception as e:
    print("Seems like the graph was already saved on the catalogue.")
    
def node_simil(tx):
    result = tx.run(
        """
        CALL gds.nodeSimilarity.stream('NSPaperTopic', {bottomK:1})
        YIELD node1, node2, similarity
        RETURN gds.util.asNode(node1).Title AS Paper1, gds.util.asNode(node2).Title AS Paper2, similarity
        ORDER BY similarity DESC
        """
    )
    records = list(result)
    summary = result.consume()
    return records, summary

with conn.driver.session(database="neo4j") as session:
    records, summary = session.execute_read(node_simil)
    print(records, summary)

Seems like the graph was already saved on the catalogue.
[<Record Paper1='Analyzing the safety impacts of raised medians' Paper2='A methodology for analyzing intersection safety' similarity=0.10526315789473684>, <Record Paper1='Hybrid power system state estimation with irregular sampling' Paper2='Probabilistic extension of flexible hybrid state estimation for cyber-physical systems' similarity=0.08695652173913043>, <Record Paper1='Influence of Communication Irregularities and Co-simulation on Hybrid Power System State Estimation' Paper2='Probabilistic extension of flexible hybrid state estimation for cyber-physical systems' similarity=0.08695652173913043>, <Record Paper1='Development of sediment management guidelines for small to medium reservoirs' Paper2='Sediment management in U.S. reservoirs: Challenges and progress [Gestion des s�diments dans les retenues am�ricaines: D�fis et progr�s]' similarity=0.08695652173913043>, <Record Paper1='Progress in transparent flat-panel holographic 